# 1. Import Libraries


In [1]:
import torch
from torch import Tensor, nn
import torch.nn.functional as F
import torchvision.models as models
from typing import Any, Optional, Tuple, Type
from torch.autograd import Variable
from torch.optim import AdamW
from torch.optim.lr_scheduler import MultiStepLR


from torchvision import transforms
import os
import gc
import time
import copy
import xml.etree.ElementTree as ET
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import InterpolationMode


import numpy as np
import pandas as pd
from scipy.integrate import simpson as simps
import math
from torchvision.models import swin_b
from tqdm import tqdm

In [2]:
# [LOG]
print(">> [1] After importing libraries!!!!")

>> [1] After importing libraries!!!!


# 2. Transformer


In [3]:
class MLPBlock(nn.Module):
    def __init__(
        self,
        embedding_dim: int,
        mlp_dim: int,
        act: Type[nn.Module] = nn.GELU,
    ) -> None:
        super().__init__()
        self.lin1 = nn.Linear(embedding_dim, mlp_dim)
        self.lin2 = nn.Linear(mlp_dim, embedding_dim)
        self.act = act()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.lin2(self.act(self.lin1(x)))


# From https://github.com/facebookresearch/detectron2/blob/main/detectron2/layers/batch_norm.py # noqa
# Itself from https://github.com/facebookresearch/ConvNeXt/blob/d1fa8f6fef0a165b27399986cc2bdacc92777e40/models/convnext.py#L119  # noqa
class LayerNorm2d(nn.Module):
    def __init__(self, num_channels: int, eps: float = 1e-6) -> None:
        super().__init__()
        self.weight = nn.Parameter(torch.ones(num_channels))
        self.bias = nn.Parameter(torch.zeros(num_channels))
        self.eps = eps

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        u = x.mean(1, keepdim=True)
        s = (x - u).pow(2).mean(1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.eps)
        x = self.weight[:, None, None] * x + self.bias[:, None, None]
        return x


class TwoWayTransformer(nn.Module):
    def __init__(
        self,
        depth: int,
        embedding_dim: int,
        num_heads: int,
        mlp_dim: int,
        activation: Type[nn.Module] = nn.ReLU,
        attention_downsample_rate: int = 2,
    ) -> None:
        """
        A transformer decoder that attends to an input image using
        queries whose positional embedding is supplied.

        Args:
          depth (int): number of layers in the transformer
          embedding_dim (int): the channel dimension for the input embeddings
          num_heads (int): the number of heads for multihead attention. Must
            divide embedding_dim
          mlp_dim (int): the channel dimension internal to the MLP block
          activation (nn.Module): the activation to use in the MLP block
        """
        super().__init__()
        self.depth = depth
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.mlp_dim = mlp_dim
        self.layers = nn.ModuleList()

        for i in range(depth):
            self.layers.append(
                TwoWayAttentionBlock(
                    embedding_dim=embedding_dim,
                    num_heads=num_heads,
                    mlp_dim=mlp_dim,
                    activation=activation,
                    attention_downsample_rate=attention_downsample_rate,
                    skip_first_layer_pe=(i == 0),
                )
            )

        self.final_attn_token_to_image = Attention(
            embedding_dim, num_heads, downsample_rate=attention_downsample_rate
        )
        self.norm_final_attn = nn.LayerNorm(embedding_dim)

    def forward(
        self,
        image_embedding: Tensor,
        image_pe: Tensor,
        point_embedding: Tensor,
    ) -> Tuple[Tensor, Tensor]:
        """
        Args:
          image_embedding (torch.Tensor): image to attend to. Should be shape
            B x embedding_dim x h x w for any h and w.
          image_pe (torch.Tensor): the positional encoding to add to the image. Must
            have the same shape as image_embedding.
          point_embedding (torch.Tensor): the embedding to add to the query points.
            Must have shape B x N_points x embedding_dim for any N_points.

        Returns:
          torch.Tensor: the processed point_embedding
          torch.Tensor: the processed image_embedding
        """
        # BxCxHxW -> BxHWxC == B x N_image_tokens x C
        bs, c, h, w = image_embedding.shape
        image_embedding = image_embedding.flatten(2).permute(0, 2, 1)
        image_pe = image_pe.flatten(2).permute(0, 2, 1)

        # Prepare queries
        queries = point_embedding
        keys = image_embedding

        # Apply transformer blocks and final layernorm
        for layer in self.layers:
            queries, keys = layer(
                queries=queries,
                keys=keys,
                query_pe=point_embedding,
                key_pe=image_pe,
            )

        # Apply the final attention layer from the points to the image
        q = queries + point_embedding
        k = keys + image_pe
        attn_out = self.final_attn_token_to_image(q=q, k=k, v=keys)
        queries = queries + attn_out
        queries = self.norm_final_attn(queries)

        return queries, keys


class TwoWayAttentionBlock(nn.Module):
    def __init__(
        self,
        embedding_dim: int,
        num_heads: int,
        mlp_dim: int = 2048,
        activation: Type[nn.Module] = nn.ReLU,
        attention_downsample_rate: int = 2,
        skip_first_layer_pe: bool = False,
    ) -> None:
        """
        A transformer block with four layers: (1) self-attention of sparse
        inputs, (2) cross attention of sparse inputs to dense inputs, (3) mlp
        block on sparse inputs, and (4) cross attention of dense inputs to sparse
        inputs.

        Arguments:
          embedding_dim (int): the channel dimension of the embeddings
          num_heads (int): the number of heads in the attention layers
          mlp_dim (int): the hidden dimension of the mlp block
          activation (nn.Module): the activation of the mlp block
          skip_first_layer_pe (bool): skip the PE on the first layer
        """
        super().__init__()
        self.self_attn = Attention(embedding_dim, num_heads)
        self.norm1 = nn.LayerNorm(embedding_dim)

        self.cross_attn_token_to_image = Attention(
            embedding_dim, num_heads, downsample_rate=attention_downsample_rate
        )
        self.norm2 = nn.LayerNorm(embedding_dim)

        self.mlp = MLPBlock(embedding_dim, mlp_dim, activation)
        self.norm3 = nn.LayerNorm(embedding_dim)

        self.norm4 = nn.LayerNorm(embedding_dim)
        self.cross_attn_image_to_token = Attention(
            embedding_dim, num_heads, downsample_rate=attention_downsample_rate
        )

        self.skip_first_layer_pe = skip_first_layer_pe

    def forward(
        self, queries: Tensor, keys: Tensor, query_pe: Tensor, key_pe: Tensor
    ) -> Tuple[Tensor, Tensor]:
        # Self attention block
        if self.skip_first_layer_pe:
            queries = self.self_attn(q=queries, k=queries, v=queries)
        else:
            q = queries + query_pe
            attn_out = self.self_attn(q=q, k=q, v=queries)
            queries = queries + attn_out
        queries = self.norm1(queries)

        # Cross attention block, tokens attending to image embedding
        q = queries + query_pe
        k = keys + key_pe
        attn_out = self.cross_attn_token_to_image(q=q, k=k, v=keys)
        queries = queries + attn_out
        queries = self.norm2(queries)

        # MLP block
        mlp_out = self.mlp(queries)
        queries = queries + mlp_out
        queries = self.norm3(queries)

        # Cross attention block, image embedding attending to tokens
        q = queries + query_pe
        k = keys + key_pe
        attn_out = self.cross_attn_image_to_token(q=k, k=q, v=queries)
        keys = keys + attn_out
        keys = self.norm4(keys)

        return queries, keys


class Attention(nn.Module):
    """
    An attention layer that allows for downscaling the size of the embedding
    after projection to queries, keys, and values.
    """

    def __init__(
        self,
        embedding_dim: int,
        num_heads: int,
        downsample_rate: int = 1,
    ) -> None:
        super().__init__()
        self.embedding_dim = embedding_dim
        self.internal_dim = embedding_dim // downsample_rate
        self.num_heads = num_heads
        assert self.internal_dim % num_heads == 0, "num_heads must divide embedding_dim."

        self.q_proj = nn.Linear(embedding_dim, self.internal_dim)
        self.k_proj = nn.Linear(embedding_dim, self.internal_dim)
        self.v_proj = nn.Linear(embedding_dim, self.internal_dim)
        self.out_proj = nn.Linear(self.internal_dim, embedding_dim)

    def _separate_heads(self, x: Tensor, num_heads: int) -> Tensor:
        b, n, c = x.shape
        x = x.reshape(b, n, num_heads, c // num_heads)
        return x.transpose(1, 2)  # B x N_heads x N_tokens x C_per_head

    def _recombine_heads(self, x: Tensor) -> Tensor:
        b, n_heads, n_tokens, c_per_head = x.shape
        x = x.transpose(1, 2)
        return x.reshape(b, n_tokens, n_heads * c_per_head)  # B x N_tokens x C

    def forward(self, q: Tensor, k: Tensor, v: Tensor) -> Tensor:
        # Input projections
        q = self.q_proj(q)
        k = self.k_proj(k)
        v = self.v_proj(v)

        # Separate into heads
        q = self._separate_heads(q, self.num_heads)
        k = self._separate_heads(k, self.num_heads)
        v = self._separate_heads(v, self.num_heads)

        # Attention
        _, _, _, c_per_head = q.shape
        attn = q @ k.permute(0, 1, 3, 2)  # B x N_heads x N_tokens x N_tokens
        attn = attn / math.sqrt(c_per_head)
        attn = torch.softmax(attn, dim=-1)

        # Get output
        out = attn @ v
        out = self._recombine_heads(out)
        out = self.out_proj(out)

        return out

In [4]:
# [LOG]
print(">> [2] After defining Transformer classes!!!!")

>> [2] After defining Transformer classes!!!!


# 3. FaceXFormer


In [5]:
class MLP(nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        output_dim: int,
        num_layers: int,
        sigmoid_output: bool = False,
    ) -> None:
        super().__init__()
        self.num_layers = num_layers
        h = [hidden_dim] * (num_layers - 1)
        self.layers = nn.ModuleList(
            nn.Linear(n, k) for n, k in zip([input_dim] + h, h + [output_dim])
        )
        self.sigmoid_output = sigmoid_output

    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = F.relu(layer(x)) if i < self.num_layers - 1 else layer(x)
        if self.sigmoid_output:
            x = F.sigmoid(x)
        return x
    
class FaceDecoder(nn.Module):
    def __init__(
        self,
        *,
        transformer_dim: 256,
        transformer: nn.Module,
        activation: Type[nn.Module] = nn.GELU,
    ) -> None:
        
        super().__init__()
        self.transformer_dim = transformer_dim
        self.transformer = transformer

        self.landmarks_token = nn.Embedding(1, transformer_dim)
        self.pose_token = nn.Embedding(1, transformer_dim)
        self.attribute_token = nn.Embedding(1, transformer_dim)
        self.visibility_token = nn.Embedding(1, transformer_dim)
        self.age_token = nn.Embedding(1, transformer_dim)
        self.gender_token = nn.Embedding(1, transformer_dim)
        self.race_token = nn.Embedding(1, transformer_dim)
        self.mask_tokens = nn.Embedding(11, transformer_dim)
        

        self.output_upscaling = nn.Sequential(
            nn.ConvTranspose2d(transformer_dim, transformer_dim // 4, kernel_size=2, stride=2),
            LayerNorm2d(transformer_dim // 4),
            activation(),
            nn.ConvTranspose2d(transformer_dim // 4, transformer_dim // 8, kernel_size=2, stride=2),
            activation(),
        )
        
        self.output_hypernetwork_mlps = MLP(
            transformer_dim, transformer_dim, transformer_dim // 8, 3
            )
                
        self.landmarks_prediction_head = MLP(
            transformer_dim, transformer_dim, 136, 3
        )
        self.pose_prediction_head = MLP(
            transformer_dim, transformer_dim, 3, 3
        )
        self.attribute_prediction_head = MLP(
            transformer_dim, transformer_dim, 40, 3
        )
        self.visibility_prediction_head = MLP(
            transformer_dim, transformer_dim, 29, 3
        )
        self.age_prediction_head = MLP(
            transformer_dim, transformer_dim, 8, 3
        )
        self.gender_prediction_head = MLP(
            transformer_dim, transformer_dim, 2, 3
        )
        self.race_prediction_head = MLP(
            transformer_dim, transformer_dim, 5, 3
        )

    def forward(
        self,
        image_embeddings: torch.Tensor,
        image_pe: torch.Tensor,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        output_tokens = torch.cat([self.landmarks_token.weight, self.pose_token.weight, self.attribute_token.weight, self.visibility_token.weight, self.age_token.weight, self.gender_token.weight, self.race_token.weight,self.mask_tokens.weight], dim=0) 
        tokens = output_tokens.unsqueeze(0).expand(image_embeddings.size(0), -1, -1)

        src = image_embeddings
        pos_src = image_pe.expand(image_embeddings.size(0), -1, -1, -1)
        b, c, h, w = src.shape

        hs, src = self.transformer(src, pos_src, tokens)
    
        landmarks_token_out = hs[:, 0, :]
        pose_token_out =  hs[:, 1, :]
        attribute_token_out = hs[:, 2, :]
        visibility_token_out = hs[:, 3, :]
        age_token_out = hs[:, 4, :]
        gender_token_out = hs[:, 5, :]
        race_token_out = hs[:, 6, :]
        mask_token_out =  hs[:, 7:, :]
        
        
        landmark_output = self.landmarks_prediction_head(landmarks_token_out)
        headpose_output = self.pose_prediction_head(pose_token_out)
        attribute_output = self.attribute_prediction_head(attribute_token_out)
        visibility_output = self.visibility_prediction_head(visibility_token_out)
        age_output = self.age_prediction_head(age_token_out)
        gender_output = self.gender_prediction_head(gender_token_out)
        race_output = self.race_prediction_head(race_token_out)
        
        src = src.transpose(1, 2).view(b, c, h, w) 
        upscaled_embedding = self.output_upscaling(src)  
        hyper_in = self.output_hypernetwork_mlps(mask_token_out)
        b, c, h, w = upscaled_embedding.shape
        seg_output = (hyper_in @ upscaled_embedding.view(b, c, h * w)).view(b, -1, h, w)
        
        
        return landmark_output, headpose_output, attribute_output, visibility_output, age_output, gender_output, race_output, seg_output



class PositionEmbeddingRandom(nn.Module):
    """
    Positional encoding using random spatial frequencies.
    """

    def __init__(self, num_pos_feats: int = 64, scale: Optional[float] = None) -> None:
        super().__init__()
        if scale is None or scale <= 0.0:
            scale = 1.0
        self.register_buffer(
            "positional_encoding_gaussian_matrix",
            scale * torch.randn((2, num_pos_feats)),
        )

    def _pe_encoding(self, coords: torch.Tensor) -> torch.Tensor:
        """Positionally encode points that are normalized to [0,1]."""
        # assuming coords are in [0, 1]^2 square and have d_1 x ... x d_n x 2 shape
        coords = 2 * coords - 1
        coords = coords @ self.positional_encoding_gaussian_matrix
        coords = 2 * np.pi * coords
        # outputs d_1 x ... x d_n x C shape
        return torch.cat([torch.sin(coords), torch.cos(coords)], dim=-1)

    def forward(self, size: Tuple[int, int]) -> torch.Tensor:
        """Generate positional encoding for a grid of the specified size."""
        h, w = size
        device: Any = self.positional_encoding_gaussian_matrix.device
        grid = torch.ones((h, w), device=device, dtype=torch.float32)
        y_embed = grid.cumsum(dim=0) - 0.5
        x_embed = grid.cumsum(dim=1) - 0.5
        y_embed = y_embed / h
        x_embed = x_embed / w

        pe = self._pe_encoding(torch.stack([x_embed, y_embed], dim=-1))
        return pe.permute(2, 0, 1)  # C x H x W

    def forward_with_coords(
        self, coords_input: torch.Tensor, image_size: Tuple[int, int]
    ) -> torch.Tensor:
        """Positionally encode points that are not normalized to [0,1]."""
        coords = coords_input.clone()
        coords[:, :, 0] = coords[:, :, 0] / image_size[1]
        coords[:, :, 1] = coords[:, :, 1] / image_size[0]
        return self._pe_encoding(coords.to(torch.float))  # B x N x C


class FaceXFormerMLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.proj = nn.Linear(input_dim, 256)

    def forward(self, hidden_states: torch.Tensor):
        hidden_states = hidden_states.flatten(2).transpose(1, 2)
        hidden_states = self.proj(hidden_states)
        return hidden_states

class FaceXFormer(nn.Module):
    def __init__(self):
        super(FaceXFormer, self).__init__()

        swin_v2 = swin_b(weights='IMAGENET1K_V1')
        self.backbone = torch.nn.Sequential(*(list(swin_v2.children())[:-1]))
        self.target_layer_names = ['0.1', '0.3', '0.5', '0.7']
        self.multi_scale_features = []
        

        embed_dim = 1024
        out_chans = 256
        
        self.pe_layer = PositionEmbeddingRandom(out_chans // 2)   

        for name, module in self.backbone.named_modules():
            if name in self.target_layer_names:
                module.register_forward_hook(self.save_features_hook(name))
        
        self.face_decoder = FaceDecoder(
            transformer_dim=256,
            transformer=TwoWayTransformer(
                depth=2,
                embedding_dim=256,
                mlp_dim=2048,
                num_heads=8,
            ))    
        
        num_encoder_blocks = 4
        hidden_sizes = [128, 256, 512, 1024]
        decoder_hidden_size = 256
        
        mlps = []
        for i in range(num_encoder_blocks):
            mlp = FaceXFormerMLP(input_dim=hidden_sizes[i])
            mlps.append(mlp)
        self.linear_c = nn.ModuleList(mlps)

        self.linear_fuse = nn.Conv2d(
            in_channels=decoder_hidden_size * num_encoder_blocks,
            out_channels=decoder_hidden_size,
            kernel_size=1,
            bias=False,
        )
    
    def save_features_hook(self, name):
        def hook(module, input, output):
            self.multi_scale_features.append(output.permute(0,3,1,2).contiguous()) 
        return hook

    def forward(self, x, labels, tasks):
        self.multi_scale_features.clear()
        
        _,_,h,w = x.shape
        features = self.backbone(x).squeeze()
        
        
        batch_size = self.multi_scale_features[-1].shape[0]
        all_hidden_states = ()
        for encoder_hidden_state, mlp in zip(self.multi_scale_features, self.linear_c):
        
            height, width = encoder_hidden_state.shape[2], encoder_hidden_state.shape[3]
            encoder_hidden_state = mlp(encoder_hidden_state)
            encoder_hidden_state = encoder_hidden_state.permute(0, 2, 1)
            encoder_hidden_state = encoder_hidden_state.reshape(batch_size, -1, height, width)
            encoder_hidden_state = nn.functional.interpolate(
                encoder_hidden_state, size=self.multi_scale_features[0].size()[2:], mode="bilinear", align_corners=False
            )
            all_hidden_states += (encoder_hidden_state,)
        
        fused_states = self.linear_fuse(torch.cat(all_hidden_states[::-1], dim=1))
        image_pe = self.pe_layer((fused_states.shape[2], fused_states.shape[3])).unsqueeze(0)
        
        landmark_output, headpose_output, attribute_output, visibility_output, age_output, gender_output, race_output, seg_output = self.face_decoder(
                image_embeddings=fused_states,
                image_pe=image_pe
            )
        
        segmentation_indices = (tasks == 0)
        seg_output = seg_output[segmentation_indices]
        
        landmarks_indices = (tasks == 1)
        landmark_output = landmark_output[landmarks_indices]

        headpose_indices = (tasks == 2)
        headpose_output = headpose_output[headpose_indices]
        
        attribute_indices = (tasks == 3)
        attribute_output = attribute_output[attribute_indices]

        age_indices = (tasks == 4)
        age_output = age_output[age_indices]
        gender_output = gender_output[age_indices]
        race_output = race_output[age_indices]
        
        visibility_indices = (tasks == 5)
        visibility_output = visibility_output[visibility_indices]
    
        # return landmark_output, headpose_output, attribute_output, visibility_output, age_output, gender_output, race_output, seg_output
        return landmark_output

In [6]:
# [LOG]
print(">> [3] After defining FaceXFormer model!!!!")

>> [3] After defining FaceXFormer model!!!!


# 4. Metrics


In [7]:
# Accuracy
class Accuracy:

    def __init__(self):
        pass

    def __repr__(self):
        return "Accuracy()"

    def test(self, label_pd, label_gt, ignore_label=-1):
        correct_cnt = 0
        total_cnt = 0
        with torch.no_grad():
            label_pd = F.softmax(label_pd, dim=1)
            label_pd = torch.max(label_pd, 1)[1]
            label_gt = label_gt.long()
            c = (label_pd == label_gt)
            correct_cnt = torch.sum(c).item()
            total_cnt = c.size(0) - torch.sum(label_gt == ignore_label).item()
        return correct_cnt, total_cnt


# FR_AUC
class FR_AUC:

    def __init__(self, data_definition):
        self.data_definition = data_definition
        if data_definition == '300W':
            self.thresh = 0.05
        else:
            self.thresh = 0.1

    def __repr__(self):
        return "FR_AUC()"

    def test(self, nmes, thres=None, step=0.0001):
        if thres is None:
            thres = self.thresh

        num_data = len(nmes)
        xs = np.arange(0, thres + step, step)
        ys = np.array([np.count_nonzero(nmes <= x)
                      for x in xs]) / float(num_data)
        fr = 1.0 - ys[-1]
        auc = simps(ys, x=xs) / thres
        return [round(fr, 4), round(auc, 6)]


# Normalized Mean Error
class NME:

    def __init__(self, nme_left_index, nme_right_index):
        self.nme_left_index = nme_left_index
        self.nme_right_index = nme_right_index

    def __repr__(self):
        return "NME()"

    def get_norm_distance(self, landmarks):
        assert isinstance(self.nme_right_index,
                          list), 'the nme_right_index is not list.'
        assert isinstance(self.nme_left_index,
                          list), 'the nme_left, index is not list.'
        right_pupil = landmarks[self.nme_right_index, :].mean(0)
        left_pupil = landmarks[self.nme_left_index, :].mean(0)
        norm_distance = np.linalg.norm(right_pupil - left_pupil)
        return norm_distance

    def test(self, label_pd, label_gt):
        nme_list = []
        label_pd = label_pd.data.cpu().numpy()
        label_gt = label_gt.data.cpu().numpy()

        for i in range(label_gt.shape[0]):
            landmarks_gt = label_gt[i]
            landmarks_pv = label_pd[i]
            if isinstance(self.nme_right_index, list):
                norm_distance = self.get_norm_distance(landmarks_gt)
            elif isinstance(self.nme_right_index, int):
                norm_distance = np.linalg.norm(
                    landmarks_gt[self.nme_left_index] - landmarks_gt[self.nme_right_index])
            else:
                raise NotImplementedError
            landmarks_delta = landmarks_pv - landmarks_gt
            nme = (np.linalg.norm(landmarks_delta, axis=1) / norm_distance).mean()
            nme_list.append(nme)
            # sum_nme += nme
            # total_cnt += 1
        return nme_list


# Count parameters in MB
def count_parameters_in_MB(model):
    if isinstance(model, nn.Module):
        return sum(v.numel() for v in model.parameters()) / 1e6
    else:
        return sum(v.numel() for v in model) / 1e6

In [8]:
# [LOG]
print(">> [4] After defining metrics!!!!")

>> [4] After defining metrics!!!!


# 5. Dataset


In [9]:
class Face300WXMLDataset(Dataset):
    """
    Dataset cho iBUG 300-W với XML annotations.
    """

    def __init__(self, root_dir, xml_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        xml_path = os.path.join(root_dir, xml_file)
        tree = ET.parse(xml_path)
        root = tree.getroot()
        for img in root.findall('images/image'):
            rel_path = img.get('file')
            img_path = os.path.join(root_dir, rel_path)
            parts = img.find('box').findall('part')
            lm = np.zeros((len(parts), 2), dtype=np.float16)
            for p in parts:
                idx = int(p.get('name'))
                lm[idx, 0] = np.float16(p.get('x'))
                lm[idx, 1] = np.float16(p.get('y'))
            self.samples.append({"img_path": img_path, "landmarks": lm})

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        image = Image.open(sample['img_path']).convert('RGB')
        landmarks = torch.from_numpy(sample['landmarks'])

        if self.transform:
            image = self.transform(image)

        return {"image": image, "landmarks": landmarks}

    # Apply the transform to all samples
    def apply_transform(self, transform) -> None:
        for sample in self.samples:
            image = Image.open(sample['img_path']).convert('RGB')
            sample['image'] = transform(image)
            sample['landmarks'] = torch.from_numpy(sample['landmarks']).float()

    # Set the transform for the dataset
    def set_transform(self, transform):
        self.transform = transform

In [10]:
# Đường dẫn dataset
if os.path.exists('./ibug_300W_large_face_landmark_dataset'):
    root = './ibug_300W_large_face_landmark_dataset'
else:
    root = '/kaggle/input/ibug-300w-large-face-landmark-dataset/ibug_300W_large_face_landmark_dataset'
test_xml = 'labels_ibug_300W_test.xml'

# Định nghĩa transform cho test
val_test_transform = transforms.Compose([
    transforms.Resize(size=(224,224), interpolation=InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
# Dataset
test_ds = Face300WXMLDataset(root, test_xml, transform=val_test_transform)

# Set transform cho train/val/test
test_ds.set_transform(val_test_transform)

# DataLoader với batch_size=48
test_loader = DataLoader(test_ds, batch_size=48, shuffle=False)

In [11]:
# [LOG]
print(">> [5] After defining dataset and dataloader!!!!")

>> [5] After defining dataset and dataloader!!!!


# 6. Test Pretrained Model


## 6.1. Configuration


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 48

In [13]:
# Print the used device
print(f"Using device: {device}")

Using device: cuda


## 6.2. Model and Metrics


In [14]:
# Load the model
model = FaceXFormer().to(device)
if os.path.exists("ckpts/facexformer_pretrained.pth"):
    weights_path = "ckpts/facexformer_pretrained.pth"
    checkpoint = torch.load(weights_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
elif os.path.exists("/kaggle/input/facexformer/pytorch/default/1/model.pt"):
    weights_path = "/kaggle/input/facexformer/pytorch/default/1/model.pt"
    checkpoint = torch.load(weights_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict_backbone'])
else:
    raise FileNotFoundError("Model weights not found. Please check the path.")
model.eval()


# Metrics
nme_metric = NME(nme_left_index=[36, 39],
                 nme_right_index=[42, 45])  # eye landmarks for normalization
# uses simps for AUC :contentReference[oaicite:3]{index=3}
fr_auc_metric = FR_AUC(data_definition='300W')

/tmp/ipykernel_84/466180601.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(weights_path, map_location=device)


In [15]:
# Enable CUDA optimization for half precision (if available)
if device.type == 'cuda':
    # Enable autocast for mixed precision training
    scaler = torch.amp.GradScaler()
    print("CUDA enabled with automatic mixed precision")
else:
    scaler = None
    print("Running on CPU, mixed precision not enabled")

# Report memory usage
if device.type == 'cuda':
    print(f"GPU memory allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
    print(f"Model size: {count_parameters_in_MB(model):.2f} MB")

CUDA enabled with automatic mixed precision
GPU memory allocated: 1.49 GB
Model size: 91.99 MB


## 6.3. Evaluation on Test Set


In [16]:
def denorm_points(points, h, w, align_corners=False):
    if align_corners:
        denorm_points = (points + 1) / 2 * torch.tensor([w - 1, h - 1], dtype=torch.float32).to(points).view(1, 1, 2)
    else:
        denorm_points = ((points + 1) * torch.tensor([w, h], dtype=torch.float32).to(points).view(1, 1, 2) - 1) / 2

    return denorm_points

In [17]:
model.eval()

# Evaluate on test
test_nmes = []
inference_times = []
with torch.no_grad():
    test_pbar = tqdm(test_loader, desc="[Testing]")
    for batch in test_pbar:
        imgs = batch['image'].to(device)
        targets = batch['landmarks'].to(device, dtype=torch.float32)

        with torch.amp.autocast(device_type=device.type, dtype=torch.float16):
            start = time.time()
            preds = model(
                imgs,
                labels=None,
                tasks=torch.tensor(
                    [1] * batch_size, dtype=torch.int8).to(device)  # landmarks
            )
            end = time.time()

            denorm_lms = denorm_points(preds.view(-1, 68, 2), 224, 224)
            test_nmes.extend(nme_metric.test(denorm_lms, targets))

        inference_times.append((end - start) / imgs.size(0))

# Metrics
mean_test_nme = sum(test_nmes) / len(test_nmes)
# [FR, AUC] using Simpson's rule :contentReference[oaicite:9]{index=9}
fr, auc = fr_auc_metric.test(test_nmes)
mean_inference_time = sum(inference_times) / len(inference_times)

print("\nTest Results:")
print(f"  Mean NME    : {mean_test_nme:.4f}")
print(f"  FR (@0.05)  : {fr:.4f}")
print(f"  AUC         : {auc:.6f}")
print(f"  Inference/s : {1/mean_inference_time:.1f} img/s")

[Testing]: 100%|██████████| 21/21 [00:40<00:00,  1.94s/it]


Test Results:
  Mean NME    : 4.3664
  FR (@0.05)  : 1.0000
  AUC         : 0.000000
  Inference/s : 137.7 img/s


In [18]:
# [LOG]
print(">> [6] After evaluating the model on test set!!!!")

>> [6] After evaluating the model on test set!!!!
